In [9]:
import numpy as np
from gallery_probes_generator import GalleryProbesGenerator

In [10]:
data_generator = GalleryProbesGenerator(path_to_dataset='data/dataset1/', probes_length=200)

gallery_names, gallery_pictures = data_generator.get_gallery()
probe_names, probe_pictures = data_generator.get_merged_probes() #changer qd la methode get_probe_total ready


Exception: gallery_names.npy does not exist, please call _generate_gallery_probes() first.

In [ ]:
norms = { "L1":lambda x: np.sum(np.abs(x)),
	  "L2":lambda x: np.sum(x**2),
	  "inf":lambda x: np.max(np.abs(x))
	}

def compute_distances_individual(data, query, norm="L2"):
	""" Compute distances.

	Computes the distances between the vectors (rows) of a dataset and a
	single query). Three distances are supported:
	  * Manhattan distance ("L1");
	  * squared Euclidean distance ("L2");
	  * Chebyshev distance ("inf").

	:param data: Dataset matrix with samples as rows.
	:param query: Query vector
	:type data: (n,d)-sized Numpy array of floats
	:type query: (d)-sized Numpy array of floats

	:result: The distances of the data vectors to the query.
	:rtype: (n)-sized Numpy array of floats
	"""
	norm_function = norms[norm]
	distances = np.zeros((len(data),), dtype=np.float32)
	for i, d in enumerate(data):
		distances[i] = norm_function(d-query)
	return distances

def compute_distance(gallery_pictures, probe_pictures, norm='L2'):
    total_array_distance = np.zeros(dtype=object, shape=200)
    for i, pic, in enumerate(probe_pictures):
        total_array_q = np.zeros(dtype=object, shape=273)
        for j, n in enumerate(gallery_pictures):
            total_array_q[j] = compute_distances_individual(n,pic)
            total_array_distance[i] = total_array_q
    return total_array_distance

In [ ]:
#test
id_probe = 0
id_gallery = 0
probe_pictures_q = probe_pictures[id_probe]
print(probe_names[id_probe])

gallery_pictures_list = gallery_pictures[id_gallery]
print(gallery_names[id_gallery])

distance_test = compute_distances_individual(gallery_pictures_list,probe_pictures_q)
print(f"moyenne main:{np.mean(distance_test)}")
print(distance_test)

In [ ]:
#distance = compute_distance(gallery_pictures,probe_pictures)
#print(distance.shape)

In [ ]:
def radius_search_bruteforce(data, q, radius, norm = 'L2'):
    distance_matrix = compute_distance(data, q, norm)
    res_all_probe = np.zeros(dtype=object, shape=200)
    for i, row in enumerate(distance_matrix):
        id_list = []
        for j, second_row in enumerate(row):
            for k, element in enumerate(second_row):
                if element <= radius:
                    id_list.append((j,k,element))
        res_all_probe[i] = id_list
    return res_all_probe

In [ ]:
resultat = radius_search_bruteforce(gallery_pictures, probe_pictures,1900000)
print(resultat)

In [ ]:
#test
i=0
tableau_150_total = np.zeros(dtype=object,shape=10)
tableau_linear_total = np.zeros(dtype=object, shape=10)
while i<10:
    tableau_150_total[i] = np.random.rand(150,150)
    tableau_linear_total[i] = np.random.rand(1,100)
    i=i+1
tableau_150_probe = np.random.rand(150,150)
tableau_linear_probe = np.random.rand(1,100)

resultat_150 = compute_distances_individual(tableau_150_total,tableau_150_probe)
resultat_linear = compute_distances_individual(tableau_linear_total,tableau_linear_probe)


In [ ]:
def search_index_in_gallery(data, name):
    id_in_gallery = None
    for i, element in enumerate(data):
        if name == element:
            id_in_gallery = i
    return id_in_gallery

In [ ]:
import matplotlib.pyplot as plt


print(probe_names[22])
index = search_index_in_gallery((gallery_names),probe_names[22])
print(index)

probe_face = plt.imshow(probe_pictures[22],cmap="gray")
plt.show()
gallery_gallery = plt.imshow(gallery_pictures[index][0],cmap="gray")
plt.show()


In [ ]:
gallery_pictures_eigen_face = np.load("data/dataset1/eigen_faces_npy/gallery_pictures_eigenface.npy", allow_pickle=True)
probe_pictures_eigen_face = np.load("data/dataset1/eigen_faces_npy/probes_merged_pictures_eigenface.npy", allow_pickle=True)
resultat_eigen = radius_search_bruteforce(gallery_pictures_eigen_face, probe_pictures_eigen_face, 18000000)
print(resultat_eigen)

In [ ]:
print(probe_names[22])
index = search_index_in_gallery(gallery_names,probe_names[22])
print(index)
distance = compute_distances_individual(gallery_pictures_eigen_face[index], probe_pictures_eigen_face[22])
print(distance)
probe_face = plt.imshow(probe_pictures[22],cmap="gray")
plt.show()
gallery_face_gallery = plt.imshow(gallery_pictures[index][0],cmap="gray")
plt.show()
print(distance.mean())

In [ ]:
print(probe_names[50])
index2 = search_index_in_gallery(gallery_names,probe_names[50])
print(index)
distance2 = compute_distances_individual(gallery_pictures_eigen_face[index2], probe_pictures_eigen_face[50])
print(distance2)
probe_face = plt.imshow(probe_pictures[50],cmap="gray")
plt.show()
gallery_face_gallery = plt.imshow(gallery_pictures[index2][0],cmap="gray")
plt.show()
print(distance2.mean())

In [ ]:
print(probe_names[150])
distance3 = compute_distances_individual(gallery_pictures_eigen_face[50], probe_pictures_eigen_face[150])
print(distance3)
probe_face = plt.imshow(probe_pictures[150],cmap="gray")
plt.show()
gallery_face_gallery = plt.imshow(gallery_pictures[50][0],cmap="gray")
plt.show()
print(distance3.mean())

In [ ]:
print(resultat_eigen[0:100])
print("separation")
print(resultat_eigen[100:200])

In [ ]:
#test adri
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

adri_total = []
adri = mpimg.imread(f'data/dataset1/adri.2.jpg')
adri_total.append(adri)

print("photo entrée : ")
adri_face = plt.imshow(adri_total[0],cmap="gray")
plt.show()
distance = compute_distance(gallery_pictures, adri_total)
#print(distance)
#resultat_adri = radius_search_bruteforce(gallery_pictures, adri_total, 1900000)

In [ ]:
from math import sqrt

def radius_opti(data):
    mean_total_list = []
    for i, row in enumerate(data):
        mean_row_list = []
        for j, element in enumerate(row):
            tmp = row
            mean_row_list.append(np.mean(compute_distances_individual(np.delete(tmp, j, axis=0), element)))
        mean_row = np.mean(mean_row_list)
        mean_total_list.append(mean_row)
    mean_total = np.mean(mean_total_list)#moyenne de distance entre deux photos d'une meme pers
    std_total = sqrt(np.var(mean_total_list))#ecart type
    return mean_total + std_total #radius opti

In [ ]:
from math import sqrt

norms = {"L1": lambda x: np.sum(np.abs(x)),
         "L2": lambda x: np.sum(x ** 2),
         "inf": lambda x: np.max(np.abs(x))
         }


def compute_distance_1d(data, query, norm="L2"):
    """Compute distances.
	Computes the distances between the vectors (rows) of a dataset and a
	single query). Three distances are supported:
	  * Manhattan distance ("L1");
	  * squared Euclidean distance ("L2");
	  * Chebyshev distance ("inf").

    :param data: (d)-sized Numpy array (1 dimension)
	:param query: Query vector
	:param norm: indicate the type of distance chosen. L1, L2, inf
	:type data: Numpy array/list of floats
	:type query: (m)-sized Numpy array of floats
    :return: (d)-sized Numpy array of all the distances between elements in data and the query
    """
    norm_function = norms[norm]
    distances = np.zeros(shape=len(data), dtype=np.float32)
    for i, d in enumerate(data):
        distances[i] = norm_function(d - query)
    return distances


def compute_distance_2d(data, q, norm='L2'):
    """Compute distances in 2D.
    Computes the distance between a dataset in 2D (array of array) and a query
    :param data: (n,d)-sized Numpy array (2 dimensions). (n) lines and (d) columns
	:param query: Query vector
	:param norm: indicate the type of distance chosen. L1, L2, inf
	:type data: Numpy array in 2 dimensions
	:type query: (m)-sized Numpy array of floats
    :return: (n,d)-sized Numpy array of all the distances between elements in a row of dataset
    and the query for each rows in dataset
    """
    total_array_q = np.zeros(dtype=object, shape=273)
    for i, person in enumerate(data):
        distances = compute_distance_1d(person, q, norm)
        total_array_q[i] = distances
    return total_array_q


In [ ]:
res = compute_distance_2d(gallery_pictures,probe_pictures[0])
print(res)